In [99]:
import sys
sys.path.append('../code')

In [100]:
import json
import cPickle as pickle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [110]:
from make_patch_df import *
from patch_nlp import get_info_from_df
from add_draft_info import *

In [102]:
hero_name_set = get_hero_names()
ability_set = get_abi_names('../../dota2-database/json/abilities.json')

In [103]:
patch_url_dict = {}
patch_url_dict['6.86'] = 'December_16,_2015_Patch'
patch_url_dict['6.85'] = 'September_24,_2015_Patch'
patch_url_dict['6.84'] = 'April_30,_2015_Patch'
patch_url_dict['6.83'] = 'December_17,_2014_Patch'
patch_url_dict['6.82'] = 'September_25,_2014_Patch'
patch_url_dict['6.81'] = 'April_29,_2014_Patch'
patch_url_dict['6.80'] = 'January_29,_2014_Patch'
patch_url_dict['6.79'] = 'October_21,_2013_Patch'
patch_url_dict['6.78'] = 'June_04,_2013_Patch'
patch_url_dict['6.77'] = 'December_19,_2012_Patch'

In [104]:
with open('../data/patch_685_df.pkl') as f:
    df_685_full = pickle.load(f)
    
# df_685 = df_from_patch_query('September_24,_2015_Patch', hero_name_set, '6.85')

In [105]:
df_685_full.head(3)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...


In [106]:
texts_685, ratios_685, labels_685 = get_info_from_df(df_685_full, labeled=True)
tfidf_train = TfidfVectorizer(use_idf=False)
tfmat_train_685 = tfidf_train.fit_transform(texts_685)
gdb_685 = GradientBoostingClassifier(random_state=0)
gdb_685.fit(tfmat_train_685.toarray(), labels_685)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=0, subsample=1.0, verbose=0, warm_start=False)

In [ ]:
all_patches = get_patch_dfs(patch_url_dict, hero_name_set, ability_set, tfidf_train, gdb_685)
all_patches['6.77'].head(3)

In [81]:
df_685 = df_685_full.copy()

In [82]:
df_685.drop(['num_buffs', 'num_nerfs'], axis=1, inplace=True)

In [83]:
counter = 0
ratios_by_hero = []
for item in df_685['text']:
    ratios_by_hero.append(ratios_685[counter:counter+len(item)])
    counter += len(item)
df_685['ratios'] = ratios_by_hero

In [84]:
ratios_685[136]

inf

In [85]:
counter = 0
for item in df_685['text']:   
    counter += len(item)
    if counter > 136:
        print item
        break

[u'Agility growth reduced from 3 to 2.6', u'Doppelganger cast time increased from 0 to 0.1']


In [86]:
# Working as intended, with inf resulting from 0.1 / 0 division. Still need to address.

In [87]:
df_685.rename(columns={'change_type':'predictions'}, inplace=True)
df_685 = df_685[list(df_685.columns[:3]) + ['text_no_abi', 'predictions', 'ratios']]
df_685.head()

,hero,patch,text,text_no_abi,predictions,ratios
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...","[cooldown reduced from 5 to 4.5, Aghanim's Sce...","[1, 1]","[0.9, 1.429]"
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,[damage increased from 12/16/20/24 to 15/20/25...,"[1, 1]","[1.25, 1.25]"
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,[cast range increased from 700 to 700/800/900/...,[1],[1.214]
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,[distance rescaled from 1000/1075/1150/1150 to...,"[-1, -1]","[0.949, 0.75]"
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],[Spark Wraith cooldown reduced from 7 to 4],[1],[0.571]


In [88]:
all_patches['6.85'] = df_685

In [89]:
with open('../data/all_patch_dfs.pkl', 'w') as f:
    pickle.dump(all_patches, f)

In [111]:
all_drafts = get_draft_dfs('../data/draft_stats/')
all_drafts = add_prev_pbs(all_drafts)
all_drafts = add_prev_wrs(all_drafts, '../data')
all_drafts = add_prev_hero_pairs(all_drafts, '../data')
all_drafts[684].tail(3)

,hero,pick%,ban%,pb%,times_pb,prev_patch_pb%,prev_patch_winrate,prev_patch_allies,prev_patch_counters
105,Riki,0.3,0.1,0.3,5,0.3,0.571,[],[]
106,Terrorblade,0.1,0.1,0.2,3,3.6,0.500,[],[]
107,Death Prophet,0.1,0.0,0.1,2,7.3,0.362,[],[]


In [91]:
[int(float(key)*100) for key in all_patches.iterkeys()]

[680, 681, 682, 683, 684, 678, 686, 677, 679, 685]

In [94]:
all_patches['6.78'].merge(all_drafts[678], how='right', on='hero').head(3)

,hero,patch,text,text_no_abi,predictions,ratios,pick%,ban%,pb%,times_pb,prev_patch_pb%,prev_patch_winrate,prev_patch_allies,prev_patch_counters
0,Abaddon,6.78,[],[],[],[],7.5,10.3,17.8,399,NaN,NaN,[],[]
1,Alchemist,6.78,[],[],[],[],26.9,30.2,57.1,1280,19.5,0.517,"[(Shadow Demon, -0.025, False), (Magnus, -0.12...","[(Rubick, -0.008, False), (Gyrocopter, -0.071,..."
2,Ancient Apparition,6.78,"[Ice Vortex cooldown decreased from 6 to 5., I...","[cooldown decreased from 6 to 5., :\n\n Base A...","[0.9358, 0.9507]","[0.833, 1.222]",2.0,0.7,2.7,60,4.0,0.388,[],[]


In [112]:
patch_draft_dfs = {}
for key, patch_df in all_patches.iteritems():
    key_as_int = int(float(key)*100)
    merged_df = patch_df.merge(all_drafts[key_as_int], how='right', on='hero')
    patch_draft_dfs[key] = merged_df

In [113]:
patch_draft_dfs['6.85'].head()

,hero,patch,text,text_no_abi,predictions,ratios,pick%,ban%,pb%,times_pb,prev_patch_pb%,prev_patch_winrate,prev_patch_allies,prev_patch_counters
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...","[cooldown reduced from 5 to 4.5, Aghanim's Sce...","[1, 1]","[0.9, 1.429]",4.0,1.3,5.3,87,2.2,0.560,[],[]
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,[damage increased from 12/16/20/24 to 15/20/25...,"[1, 1]","[1.25, 1.25]",15.8,23.2,39.0,640,1.4,0.500,[],[]
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,[cast range increased from 700 to 700/800/900/...,[1],[1.214],19.8,26.7,46.4,761,25.0,0.469,"[(Clockwerk, 0.02, False), (Rubick, -0.069, Fa...","[(Earthshaker, 0.017, False), (Gyrocopter, 0.1..."
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,[distance rescaled from 1000/1075/1150/1150 to...,"[-1, -1]","[0.949, 0.75]",7.7,12.3,20.1,329,24.2,0.526,"[(Lina, -0.061, False), (Earthshaker, 0.0, Fal...","[(Earthshaker, -0.038, False), (Dark Seer, -0...."
4,Axe,6.85,[],[],[],[],2.9,2.7,5.6,91,10.8,0.478,[],[]


In [114]:
with open('../data/all_patch_draft_dfs.pkl', 'w') as f:
    pickle.dump(patch_draft_dfs, f)